In [1]:
import pandas as pd
import os
import json
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# !pip install dash plotly

In [3]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [4]:
def get_demography_df():
    files = os.listdir("data/demography/")
    entries = []
    for file in tqdm(files):
        with open(f"data/demography/{file}","r") as f:
            entry = json.load(f)
            entry["frame_id"] = int(file.split("_")[1])
            entries.append(entry)
    demography_df = pd.DataFrame(entries)
    return demography_df

In [5]:
demo_df = get_demography_df()

100%|█████████████████████████████████████████████████████████████████████████████| 1127/1127 [00:02<00:00, 541.36it/s]


In [37]:
demo_df.head()

,emotion,dominant_emotion,region,face_confidence,age,gender,dominant_gender,race,dominant_race,frame_id
0,"{'angry': 0.07739887564168878, 'disgust': 2.20...",sad,"{'x': 191, 'y': 157, 'w': 168, 'h': 292, 'left...",1.00,31,"{'Woman': 22.41971045732498, 'Man': 77.5802910...",Man,"{'asian': 1.7354777289006387, 'indian': 1.1071...",white,10005
1,"{'angry': 0.05867492408528019, 'disgust': 9.89...",sad,"{'x': 196, 'y': 138, 'w': 163, 'h': 295, 'left...",1.00,30,"{'Woman': 16.770778596401215, 'Man': 83.229219...",Man,"{'asian': 4.841800034046173, 'indian': 3.40968...",white,10020
2,"{'angry': 3.0267726747068555, 'disgust': 2.485...",neutral,"{'x': 205, 'y': 115, 'w': 163, 'h': 276, 'left...",1.00,31,"{'Woman': 13.677968084812164, 'Man': 86.322033...",Man,"{'asian': 1.998317427933216, 'indian': 1.90386...",white,10035
3,"{'angry': 1.7797511020489765e-06, 'disgust': 1...",neutral,"{'x': 344, 'y': 80, 'w': 153, 'h': 288, 'left_...",0.98,33,"{'Woman': 93.70995759963989, 'Man': 6.29003867...",Woman,"{'asian': 0.13043524231761694, 'indian': 0.189...",white,10050
4,"{'angry': 17.719951272010803, 'disgust': 7.726...",neutral,"{'x': 349, 'y': 72, 'w': 153, 'h': 304, 'left_...",0.99,33,"{'Woman': 95.17351388931274, 'Man': 4.82648648...",Woman,"{'asian': 1.2738639488816261, 'indian': 2.4188...",white,10065


In [17]:
# Divide into age-categories
age_list = [0, 18, 25, 45, 65]
def map_age(age):
    for idx, upper in enumerate(age_list):
        if age < upper:
            return f"{age_list[idx-1]}-{upper-1}"
    return "65+"
demo_df["age_group"] = demo_df.age.apply(map_age)

In [18]:
demo_df.head()

,emotion,dominant_emotion,region,face_confidence,age,gender,dominant_gender,race,dominant_race,frame_id,age_group
0,"{'angry': 0.07739887564168878, 'disgust': 2.20...",sad,"{'x': 191, 'y': 157, 'w': 168, 'h': 292, 'left...",1.00,31,"{'Woman': 22.41971045732498, 'Man': 77.5802910...",Man,"{'asian': 1.7354777289006387, 'indian': 1.1071...",white,10005,25-44
1,"{'angry': 0.05867492408528019, 'disgust': 9.89...",sad,"{'x': 196, 'y': 138, 'w': 163, 'h': 295, 'left...",1.00,30,"{'Woman': 16.770778596401215, 'Man': 83.229219...",Man,"{'asian': 4.841800034046173, 'indian': 3.40968...",white,10020,25-44
2,"{'angry': 3.0267726747068555, 'disgust': 2.485...",neutral,"{'x': 205, 'y': 115, 'w': 163, 'h': 276, 'left...",1.00,31,"{'Woman': 13.677968084812164, 'Man': 86.322033...",Man,"{'asian': 1.998317427933216, 'indian': 1.90386...",white,10035,25-44
3,"{'angry': 1.7797511020489765e-06, 'disgust': 1...",neutral,"{'x': 344, 'y': 80, 'w': 153, 'h': 288, 'left_...",0.98,33,"{'Woman': 93.70995759963989, 'Man': 6.29003867...",Woman,"{'asian': 0.13043524231761694, 'indian': 0.189...",white,10050,25-44
4,"{'angry': 17.719951272010803, 'disgust': 7.726...",neutral,"{'x': 349, 'y': 72, 'w': 153, 'h': 304, 'left_...",0.99,33,"{'Woman': 95.17351388931274, 'Man': 4.82648648...",Woman,"{'asian': 1.2738639488816261, 'indian': 2.4188...",white,10065,25-44


In [38]:
race_emotion_count = demo_df.groupby(['dominant_race', 'dominant_emotion']).size().reset_index(name='count')
# Calculate the total count for normalization
total_count = race_emotion_count['count'].sum()

# Calculate percentages
race_emotion_count['percentage'] = (race_emotion_count['count'] / total_count) * 100
race_emotion_count.head()

,dominant_race,dominant_emotion,count,percentage
0,asian,angry,4,0.354925
1,asian,fear,12,1.064774
2,asian,happy,7,0.621118
3,asian,neutral,9,0.798580
4,asian,sad,71,6.299911


In [39]:
# Get unique lists of races and emotions
unique_races = race_emotion_count['dominant_race'].unique().tolist()
unique_emotions = race_emotion_count['dominant_emotion'].unique().tolist()

# Create a mapping of race and emotion to index
race_to_index = {race: i for i, race in enumerate(unique_races)}
emotion_to_index = {emotion: i + len(unique_races) for i, emotion in enumerate(unique_emotions)}

# Apply mapping to get sources and targets
race_emotion_count['source'] = race_emotion_count['dominant_race'].map(race_to_index)
race_emotion_count['target'] = race_emotion_count['dominant_emotion'].map(emotion_to_index)


In [40]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
      pad=15,
      thickness=20,
      line=dict(color="black", width=0.5),
      label=unique_races + unique_emotions,
      color="blue"
    ),
    link=dict(
      source=race_emotion_count['source'].tolist(),
      target=race_emotion_count['target'].tolist(),
      value=race_emotion_count['percentage'].tolist()
    ))])

fig.update_layout(title_text="Sankey Diagram: Race and Emotion Flow in Movie", font_size=10)
fig.show()

In [20]:
from dash import Dash, dcc, html
# import dash_mantine_components as dmc
from dash.dependencies import Input, Output

In [120]:
app = Dash()

In [121]:
def get_filtered_df(selected_range):
    # Compute min and max frame_id
    min_frame_id = demo_df['frame_id'].min()
    max_frame_id = demo_df['frame_id'].max()
    range_frame_id = max_frame_id - min_frame_id
    
    # Convert percentage slider values to frame_id range
    start_id = min_frame_id + (selected_range[0] / 100.0) * range_frame_id
    end_id = min_frame_id + (selected_range[1] / 100.0) * range_frame_id
    
    # Filter the DataFrame based on the calculated frame_id range
    filtered_df = demo_df[(demo_df['frame_id'] >= start_id) & (demo_df['frame_id'] <= end_id)]
    return filtered_df

In [122]:
group_order = ['dominant_race', 'dominant_emotion', 'age_group']
filtered_df = demo_df
transition_count = filtered_df.groupby(group_order).size().reset_index(name='count')
total_count = transition_count['count'].sum()
transition_count['percentage'] = (transition_count['count'] / total_count) * 100
transition_count.head()
labels = [filtered_df[col].unique() for col in group_order]
labels_flat = [item for sublist in labels for item in sublist]
index_mapping = {label: idx for idx, label in enumerate(labels_flat)}
# Map the labels to indices
transition_count['source'] = transition_count[group_order[0]].map(index_mapping)
transition_count['target1'] = transition_count[group_order[1]].map(index_mapping)
transition_count['target2'] = transition_count[group_order[2]].map(index_mapping)
transition_count.head()

,dominant_race,dominant_emotion,age_group,count,percentage,source,target1,target2
0,asian,angry,25-44,4,0.354925,1,7,11
1,asian,fear,25-44,12,1.064774,1,10,11
2,asian,happy,25-44,7,0.621118,1,8,11
3,asian,neutral,25-44,9,0.798580,1,6,11
4,asian,sad,25-44,70,6.211180,1,5,11


In [123]:
# Callback to update the sankey diagram based on the slider
@app.callback(
    Output('sankey-emotion-race-age-graph', 'figure'),
    Input('frame-slider', 'value'),
    Input('race-emotion-age-radio-btn', 'value'))
def update_emotion_race_sankey_graph_slider(selected_range, col_radio):
    filtered_df = get_filtered_df(selected_range)
    if col_radio == 'dominant_race':
        group_order = ['dominant_race', 'dominant_emotion', 'age_group']
    elif col_radio == 'dominant_emotion':
        group_order = ['dominant_emotion', 'age_group', 'dominant_race']
    else:  # 'age-race-emotion'
        group_order = ['age_group', 'dominant_race', 'dominant_emotion']

    # Group and count transitions
    transition_count = filtered_df.groupby(group_order).size().reset_index(name='count')
    first_transition = filtered_df.groupby(group_order[:2]).size().reset_index(name='count1')
    second_transition = filtered_df.groupby(group_order[1:]).size().reset_index(name='count2')

    # Calculate total counts for normalization of each segment
    total_first = first_transition['count1'].sum()
    total_second = second_transition['count2'].sum()

    first_transition['percentage'] = (first_transition['count1'] / total_first) * 100
    second_transition['percentage'] = (second_transition['count2'] / total_second) * 100

    # Get unique labels for nodes and create mappings
    labels = [filtered_df[col].unique() for col in group_order]
    labels_flat = [item for sublist in labels for item in sublist]
    index_mapping = {label: idx for idx, label in enumerate(labels_flat)}

    # Map the labels to indices for both transitions
    first_transition['source'] = first_transition[group_order[0]].map(index_mapping)
    first_transition['target'] = first_transition[group_order[1]].map(index_mapping)

    second_transition['source'] = second_transition[group_order[1]].map(index_mapping)
    second_transition['target'] = second_transition[group_order[2]].map(index_mapping)

    # Create the Sankey diagram data structure
    node_labels = labels_flat
    links = {
        'source': first_transition['source'].tolist() + second_transition['source'].tolist(),
        'target': first_transition['target'].tolist() + second_transition['target'].tolist(),
        'value': first_transition['percentage'].tolist() + second_transition['percentage'].tolist(),
    }

    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_labels,
            color="blue"
        ),
        link=links
    )])

    fig.update_layout(title_text="Sankey Diagram: Dynamic Transitions", font_size=10)
    return fig


In [124]:
# Callback to update the gender race/emotion/age bar graph based on the slider
@app.callback(
    Output('gender-emotion-race-age-bar-graph', 'figure'),
    Input('race-emotion-age-radio-btn', 'value'),
    Input('frame-slider', 'value'))
def update_gender_emotion_race_age_graph_slider(col_radio, selected_range):
    filtered_df = get_filtered_df(selected_range)
    gender_df = filtered_df.groupby([col_radio,"dominant_gender"]).size().unstack(fill_value=0)
    grouped_percent = gender_df.div(gender_df.sum(axis=1), axis=0) * 100
    title_ref = {"dominant_race": "Race", "dominant_emotion": "Emotion", "age_group": "Age"}
    title_val = title_ref[col_radio]
    # Plotting the stacked bar chart
    fig = go.Figure()
    
    # Add one trace for each gender
    for gender in grouped_percent.columns:
        fig.add_trace(go.Bar(
            name=gender,
            x=grouped_percent[gender],  # percentage values
            y=grouped_percent.index,  # race/emotion categories
            orientation='h'
        ))
    
    # Update layout for a stacked bar chart
    fig.update_layout(
        barmode='stack',
        title=f'Distribution of Gender by {title_val}',
        xaxis_title=f'Percentage of {title_val}',
        yaxis_title=f'{title_val}',
    )
    
    return fig

In [125]:
# Callback to update the spider graph based on the slider
@app.callback(
    Output('spider-graph', 'figure'),
    Input('frame-slider', 'value'),
    Input('race-emotion-age-radio-btn', 'value'))
def update_radial_graph_slider(selected_range, col_radio):
    filtered_df = get_filtered_df(selected_range)
    
    specific_df = pd.DataFrame(filtered_df[col_radio].value_counts(normalize=True)*100).reset_index()
    # Create the figure
    fig = go.Figure(data=go.Scatterpolar(
        r=specific_df['proportion'],
        theta=specific_df[col_radio],
        fill='toself'
    ))

    fig.update_layout(
        title="Percentage representation in movie segment",
        polar=dict(
            radialaxis=dict(
                visible=True
            ),
        ),
        showlegend=False
    )
    
    return fig

In [126]:
# Callback to update the line graph based on the slider
@app.callback(
    Output('evolution-line-graph', 'figure'),
    Input('frame-slider', 'value'),
    Input('race-emotion-age-radio-btn', 'value'))
def update_line_graph_slider(selected_range, col_radio):
    filtered_df = get_filtered_df(selected_range)
    filtered_df.loc[:,"count"] = 1
    pivot_df = filtered_df.pivot_table(index="frame_id", columns=col_radio, values="count", aggfunc="sum", fill_value=0).cumsum()
    # Create the figure
    fig = go.Figure()

    # Make sure the columns are treated as a list even if there's only one
    columns_list = pivot_df.columns.tolist()
    
    # Adding traces for each race/emotion/age
    for column in columns_list:
        fig.add_trace(go.Scatter(
            x=pivot_df.index,
            y=pivot_df[column],
            mode='lines',
            name=str(column),
            hoverinfo='name+x+y',
            stackgroup='one' # Stacking
        ))
    
    # updating the layout
    fig.update_layout(
        title= "Representation over time in movie segment",
        xaxis_title="Frame ID",
        yaxis_title="Cumulative count of appearances",
        hovermode="x"
    )
    
    return fig

In [127]:
app.layout = html.Div([
    html.H1(children="Movie Visualizations"),
    dcc.RangeSlider(
        id='frame-slider',
        min=0,
        max=100,
        value=[0, 100],
        step=5,  # Percentage step
        marks={i: f'{i}%' for i in range(0, 101, 10)}
    ),
    html.H2(children="Distribution of Race, Emotion and Age"),
    html.Div([
        dcc.RadioItems(
            id='race-emotion-age-radio-btn',
            options=[
                {'label': 'Race', 'value': 'dominant_race'},
                {'label': 'Emotion', 'value': 'dominant_emotion'},
                {'label': 'Age', 'value': 'age_group'}
            ],
            value='dominant_race',  # Default value
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}  # Space out the buttons
        )
    ], style={'margin-bottom': '20px'}),  # Space below the radio buttons
    # Container for graphs
    html.Div([
        dcc.Graph(id='spider-graph', style={'display': 'inline-block', 'width': '50%'}),
        dcc.Graph(id='evolution-line-graph', style={'display': 'inline-block', 'width': '50%'})
    ], style={'display': 'flex'}),
    html.Div([
        dcc.Graph(id='gender-emotion-race-age-bar-graph', style={'display': 'inline-block', 'width': '50%'}),
        dcc.Graph(id='sankey-emotion-race-age-graph', style={'display': 'inline-block', 'width': '50%'})
    ], style={'display': 'flex'})
])

In [128]:
app.run(debug=True)

C:\Users\Debadyuti\AppData\Local\Temp\ipykernel_4484\374672561.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Debadyuti\AppData\Local\Temp\ipykernel_4484\374672561.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Debadyuti\AppData\Local\Temp\ipykernel_4484\374672561.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org